# Tutorial 3

**In this tutorial you will:**
- Build a Chatbot connected to a code interpreter able to execute Python code [Section 1](#1-chatbotwithinterpreter-expanding-interactivity-and-functionality)

In [ ]:
%load_ext autoreload
%autoreload 2
#imports
from aiflows.utils.general_helpers import read_yaml_file, quick_load_api_keys
from aiflows.backends.api_info import ApiInfo
from aiflows.utils import serving
from aiflows.utils import colink_utils
from aiflows.workers import run_dispatch_worker_thread
from aiflows.base_flows import AtomicFlow
from aiflows.messages import FlowMessage
from aiflows import flow_verse
import sys
import os
sys.path.append("..")
from utils import compile_and_writefile, dict_to_yaml
import json
import copy
#Specify path of your flow modules


In [ ]:
#starting a local colink server
cl = colink_utils.start_colink_server()

In [ ]:
# Start Worker thread
run_dispatch_worker_thread(cl)

In [ ]:
# Start 2nd Worker thread (in case you're making blocking calls)
run_dispatch_worker_thread(cl)

## 1. ChatBotWithInterpreter: Expanding Interactivity and Functionality

In this section, we'll explore the implementation of a chatbot integrated with a code interpreter, offering new dimensions of interactivity and functionality. By connecting an interpreter capable of compiling and executing code to the chatbot, we unlock numerous benefits. This setup enables users to prompt the chatbot with specific tasks, allowing them to provide instructions like "download Apple's stock prices, plot them, and save them in a PDF." In response, the chatbot generates tailored code to execute these tasks, and executes it. 

Additionally, integrating the interpreter enhances the chatbot's adaptability and versatility. By understanding the user's environment, such as their Python environment, the chatbot can execute code seamlessly within that context. This capability extends to installing necessary Python packages on-the-fly, ensuring smooth execution without requiring user intervention. Such integration enhances the chatbot's utility, making it a versatile tool for coding assistance, troubleshooting, and rapid prototyping. Furthermore, this integration ensures that all generated code is compilable, maintaining a seamless user experience and efficient task execution.


### 1.1 Leveraging ChatAtomicFlow to write a CodeGenerator Flow

##### Pulling ChatAtomicFlow from FlowVerse

In [ ]:
#Install dependencies from FlowVerse
dependencies = [
    {"url": "aiflows/ChatFlowModule", "revision": "main"},
]
from aiflows import flow_verse
flow_verse.sync_dependencies(dependencies)

##### Writing the CodeGenerator's Default Config

In [ ]:
#Defining prompt templates

#system prompt template
system_prompt_template = \
"""
You are a world class programmer that can complete any goal with code.
      
Your tasks are:
    1. Write a python code in order to achieve a goal that is given to you. This code should be able to be run by the executor.

Notice that:
    1. If you use any external libraries, you must install them withing the code (in python)
    2. All functions you write are modular, so make sure you make imports necessary imports within the function.
    3. You must write docstrings for every function you write.

    
Your function will then be imported and called by an executor to finish the goal, you do not need to worry about the execution part.
The executor will give you feedback on the code you write, and you can revise your code based on the feedback.

Performance Evaluation:
1. You must write your code in python
2. Your answer must be able to be compiled and run by the executor (i.e, do not write codeblocks)

**It's important that you should only respond in JSON format as described below:**
Response Format:
{
    "language_of_code": "language of the code",
    "code": "String of the code and docstrings corresponding to the goal",
    "finish": "True if you have finished the goal, False otherwise. Take in consideration the feedback you receive to decide if you have finished the goal or not",
}
Ensure your responses can be parsed by Python json.loads
    
**It's important that the code you generate can be written by Python write, and is human-readable. The written file must also be indented and formatted, so that it is human-readable.**
"""

#init_human_prompt_template
#This is the prompt passed to the ChatAtomicFlow the first time its called (i.e., the user asks to accomplish a goal)
init_human_prompt_template = \
"""
"Here is the goal you need to achieve:
{{goal}}"
"""

#human_prompt_template
#This is the prompt passed to the ChatAtomicFlow after the first time its called (i.e., feedback from the code interpreter)
human_prompt_template = \
"""
Here is the goal you need to achieve:
{{goal}}
Here is the previous code you have written:
{{previous_code}}
Here is the feedback from the previous code:
{{feedback}}
"""

In [ ]:
default_config_code_generator = {
    "_target_": "CodeGeneratorFlowModule.CodeGenerator.CodeGenerator.instantiate_from_default_config",
    "name": "CodeGenerator",
    "description": "Writes code with given instruction",
    "backend": {
        "api_infos": "???"
    },
    "model_name": {
        "openai": "gpt-4"
    },
    "input_interface_non_initialized": ["goal"],

    "input_interface_initialized": ["goal", "previous_code", "feedback"],

    "output_interface": ["language_of_code", "code"],

    "system_message_prompt_template": {
        "template": f'{system_prompt_template}'
    },
  
    "human_message_prompt_template": {
        "template": f'{human_prompt_template}',
        "input_variables": ["goal", "previous_code", "feedback"],
    },

    "init_human_message_prompt_template": {
        "template": f'{init_human_prompt_template}',
        "input_variables": ["goal"]
    },
    #keep as context window only the first message (system prompt template) and the last two messages (last user-assistant interaction)
    "previous_messages": {
        "first_k": 1,
        "last_k": 2,
    }
}
dict_to_yaml(default_config_code_generator, "CodeGeneratorFlowModule/CodeGenerator.yaml")

##### Writing the CodeGenerator Flow

**Requirements:** We want to make sure that the CodeGenerator generates a json parsable output. Implement this in the `CodeGenerator` class by requerying the LLM if it doesn't generate a parsable output (specifying to reformat its previous output).

In [ ]:
%%compile_and_writefile CodeGeneratorFlowModule/CodeGenerator.py

from aiflows.base_flows import AtomicFlow
from aiflows.messages import FlowMessage
from flow_modules.aiflows.ChatFlowModule import ChatAtomicFlow
import json


class CodeGenerator(ChatAtomicFlow):

    def run(self, input_message: FlowMessage):
        input_data = input_message.data
        json_parsable = False
        response = None
        
        #ensure the response is json parsable
        while not json_parsable:
            
            output = self.query_llm(input_data=input_data).strip()
            
            try:
                response = json.loads(output)
                json_parsable = True
            
            except (json.decoder.JSONDecodeError, json.JSONDecodeError):
                
                ###### TODO: IF OUTPUT IN NOT JSON PARSABLE, REQUERY THE LLM 
                # HINT: Changing the input_data with feddback on the non json parsable response is a good way to go
                ## Check out config above to see what input_data should have (you are querying the LLM for the second time (at least))
                ## so look at human_message_prompt_template
        ##package output message
        reply = ???
        ### send back reply
        ???

##### Serving the CodeGenerator Flow:

In [ ]:
serving.serve_flow(
    cl=cl,
    flow_class_name="CodeGeneratorFlowModule.CodeGenerator.CodeGenerator",
    flow_endpoint="CodeGenerator",
)

##### Testing the CodeGenerator Flow:

We will now test the CodeGenerator Flow by prompting the chatbot with a task and observing the generated code. We will use the following task as an example: "Download Apple's stock prices, plot them, and save them in a PDF." The chatbot should generate code that accomplishes this task, and we will evaluate the generated code's quality and correctness. To showcase the limitations of using the code generator on its own, we will show that the code is not always compilable in your Python environment. To showcase we will manually uninstall, if you do have it installed, the `yfinance` package from the Python environment (which is a library the Flow will use in its answer)

In [ ]:
!pip uninstall yfinance -y

In [ ]:

path_to_yaml = os.path.join("CodeGeneratorFlowModule", "CodeGenerator.yaml")
default_config = read_yaml_file(path_to_yaml)
overrides = copy.deepcopy(default_config)
api_information = [ApiInfo(backend_used="openai", api_key=os.getenv("OPENAI_API_KEY"))]
quick_load_api_keys(overrides, api_information)

proxy_code_generator = serving.get_flow_instance(
    cl=cl,
    flow_endpoint="CodeGenerator", #TODO: SPECIFY THE ENDPOINT
    user_id="local",
    config_overrides=overrides,
)
data = {
    "goal": "Download Apple's stock price between 2015 and 2016, make a plot with it and save it as a pdf in './apple_stocks.pdf'",
}
input_message = proxy_code_generator.package_input_message(data)
future = proxy_code_generator.get_reply_future(input_message)
reply_data = future.get_data()
print("Data sent:\n",  data, "\n")
print(f'REPLY:\n{reply_data["code"]} \n')


Now if you copy paste the reply from the chatbot, you will see that the code is not compilable. This is because the `yfinance` package is not installed in your Python environment. This is an example where the interpreter comes in handy.

In [ ]:
#### COPY PASTE YOU LLMS REPLY TO SEE IT DOES NOT RUN

### 1.2 Using the Interperter Flow Module

We already have an interpreter flow module implemented on the flow verse which you can checkout [here](https://huggingface.co/aiflows/InterpreterFlowModule) !

Let's check out its config, serve it and test it.

##### Pulling the InterpreterFlowModule from FlowVerse



In [ ]:
#Install dependencies from FlowVerse
dependencies = [
    {"url": "aiflows/InterpreterFlowModule", "revision": "main"},
]
from aiflows import flow_verse
flow_verse.sync_dependencies(dependencies)


In [ ]:
#print it's default config

interpreter_default_config = read_yaml_file("flow_modules/aiflows/InterpreterFlowModule/InterpreterAtomicFlow.yaml")
print(json.dumps(interpreter_default_config, indent=4))

##### Serving the InterpreterFlowModule

In [ ]:
serving.serve_flow(
    cl=cl,
    flow_class_name="flow_modules.aiflows.InterpreterFlowModule.InterpreterAtomicFlow",
    flow_endpoint="InterpreterAtomicFlow",
)
run_dispatch_worker_thread(cl)

##### Testing the InterpreterFlowModule

In [ ]:
proxy_code_interpreter = serving.get_flow_instance(
    cl=cl,
    flow_endpoint="InterpreterAtomicFlow", #TODO: SPECIFY THE ENDPOINT
    user_id="local",
)
data = [
    #should return "Hello, World!"
    {"language": "Python", "code": "print('Hello, World!')"},
    #should return an error
    {"language": "Python", "code": "import yfinance as yf\nyf.download('AAPL', start='2015-01-01', end='2016-01-01')"},
]

for dp in data:
    input_message = proxy_code_interpreter.package_input_message(dp)
    future = proxy_code_interpreter.get_reply_future(input_message)
    reply_data = future.get_data()
    print("Data sent:\n",  dp, "\n")
    print("REPLY:\n", reply_data, "\n")

### 1.3 Combining the CodeGenerator and Interpreter Flow Modules (ChatCodeInterpreter Flow)

Now that we have both the CodeGenerator and Interpreter Flow Modules, we can combine them to create a ChatCodeInterpreter Flow. This flow will leverage the CodeGenerator to generate code based on user prompts, and the Interpreter to execute the generated code. The CodeGenerator will decide when to finish generating the code and return the final output (see system prompt template)

##### Writing the default config of the ChatCodeInterpreter Flow

In [ ]:
init_human_prompt_template = \
"""
Here is the goal you need to achieve:
{{goal}}"
Feedback:
Do not finish this goal yet. You will receive feedback on the code you write.
"""


In [ ]:
default_config_ChatCodeInterpreter = \
{
    "name": "ChatCodeInterpreter",
    "description": "A code which generate code with the help of a interpreter",

    # TODO: Define the target
    "_target_": "ChatCodeInterpreterFlowModule.ChatCodeInterpreter.ChatCodeInterpreter.instantiate_from_default_config",

    "input_interface": "goal",
    "output_interface": ["code","interpreter_output"],
    
    "subflows_config": {
        "Coder": {
            "user_id": "local",
            "flow_endpoint": "CodeGenerator",
            "name": "Proxy of Coder",
            "description": "A proxy flow of the Coder flow.",
             "backend": {
                "api_infos": "???",
                "model_name": {"openai": "gpt-4"}
            },
             "init_human_message_prompt_template": {
                "template": f'{init_human_prompt_template}',
                "input_variables": ["goal"]
             }
        },
        "Interpreter": {
           
            "user_id": "local",
            
            "flow_endpoint": "InterpreterAtomicFlow",
            "name": "Proxy of Interpreter Flow",
            "description": "A proxy flow of the Interpreter Flow.",
        }
    }
}
dict_to_yaml(default_config_ChatCodeInterpreter, "ChatCodeInterpreterFlowModule/ChatCodeInterpreter.yaml")

##### Writing the ChatCodeInterpreter Flow


In [ ]:
%%compile_and_writefile ChatCodeInterpreterFlowModule/ChatCodeInterpreter.py


######## TODO WRITE THE ChatCodeInterpreter Class #######
####### HINT: TAKE INSPIRATION FROM THE PREVIOUS TUTORIAL 
####### (4.ChatWithGuards --> The ChatWithPIGuard and ChatQueryDBwithPIGuard Flows)
from aiflows.base_flows import CompositeFlow
from aiflows.messages import FlowMessage
from aiflows.interfaces import KeyInterface

class ChatCodeInterpreter(CompositeFlow):
        
    def __init__(self,**kwargs):
        ???
        
    def set_up_flow_state(self):
        """ Set up the flow state. (called in __init__)"""
        super().set_up_flow_state()
        
    def determine_current_state(self):
        """ Determine the current state of the flow."""
        ???
                        
    def call_coder(self):
        """ Call the coder subflow."""
        ???
        
    def call_interpreter(self):
        """ Call the interpreter subflow."""
        ???
        
    def generate_reply(self):
        """ Answers back to the original input message"""  
        ???
        
    def register_data_to_state(self, input_message):
        """ Register the data from the input message to the flow state."""
        ????

   
    def run(self, input_message: FlowMessage):
        """ Run the flow."""
        ???

##### Serve the ChatCodeInterpreter Flow

In [ ]:
serving.serve_flow(
    cl=cl,
    flow_class_name="ChatCodeInterpreterFlowModule.ChatCodeInterpreter.ChatCodeInterpreter",
    flow_endpoint="ChatCodeInterpreter",
)

run_dispatch_worker_thread(cl)

In [ ]:
#make sure we don't have yfinacne installed
!pip uninstall yfinance -y

##### Testing the ChatCodeInterpreter Flow

Now let's test the ChatCodeInterpreter Flow by prompting the chatbot with the same task as before: "Download Apple's stock prices, plot them, and save them in a PDF." The chatbot should generate code that accomplishes this task, and the interpreter should execute the generated code (including installing the necessary packages).

In [ ]:

default_config = default_config_ChatCodeInterpreter
overrides = copy.deepcopy(default_config)
api_information = [ApiInfo(backend_used="openai", api_key=os.getenv("OPENAI_API_KEY"))]
quick_load_api_keys(overrides, api_information)

proxy_code_generator = serving.get_flow_instance(
    cl=cl,
    flow_endpoint="ChatCodeInterpreter", #TODO: SPECIFY THE ENDPOINT
    user_id="local",
    config_overrides=overrides,
)
data = [
    {"id": 0, "goal": "Download Apple's stock price between 2015 and 2016, make a plot with it, \
        and save it as a pdf in Download Apple's stock price between 2015 and 2016, make a plot with \
            it and save it as a pdf in 'apple_stocks.pdf'"},
]

for dp in data:
    input_message = proxy_code_generator.package_input_message(dp)
    future = proxy_code_generator.get_reply_future(input_message)
    reply_data = future.get_data()
    print("Data sent:\n",  dp, "\n")
    print("REPLY:\n", reply_data, "\n")

Note how a pdf named `apple_stock_prices.pdf` is saved in the current directory. This is the pdf that was generated by the ChatAtomicFlow executerd by the interpreter Flow.

You can also check out the code it generated:

In [ ]:
print(reply_data["code"])

If you copy paste the code in a new cell it should also run since it installed the necessary packages !

In [ ]:
### COPY PASTE YOUR CODE (IT SHOULD RUN)